In [1]:
import os

# Subir un nivel respecto a la carpeta actual
os.chdir("..")

In [2]:
from Utils import load_parameters_file, from_matrix_to_preset, MSE, denormalize_preset, normalize_preset, pretty_print, PARAM_NAMES, from_preset_to_matrix, mel_spectrogram, mfcc
from Synth import Synth
import numpy as np
import cma
import matplotlib.pyplot as plt
from multiprocessing import Pool
from parallelEvaluation import evaluate_presets
from globals import SAMPLE_RATE, DURATION, PROCESSORS
from IPython.display import Audio

In [3]:
def render_presets(presets):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=presets
    )
    return synth.process_audio()

def evaluate_target(preset):
    synth = Synth(
        sample_rate=SAMPLE_RATE,
        duration=DURATION,
        presets=preset
    )
    audio = synth.process_audio()
    # return mfcc(audio, sr=SAMPLE_RATE)
    return mel_spectrogram(audio, sr=SAMPLE_RATE)

In [4]:
target_audio = render_presets(load_parameters_file('target.json'))
Audio(target_audio[0], rate=SAMPLE_RATE)

In [5]:
target_C = evaluate_target(load_parameters_file('target.json'))

def get_fitness(solutions):
    solutions_splitted = np.split(solutions, PROCESSORS)
    presets_splitted = [(denormalize_preset(from_matrix_to_preset(chunk)), target_C) for chunk in solutions_splitted]
    
    solutions_evaluated = pool.map(evaluate_presets, presets_splitted)
    return np.concatenate(solutions_evaluated)

# def evaluate_single(x):
#     """Función que evalúa un solo individuo (requerida por NoiseHandler)."""
#     # print(x)
#     preset = denormalize_preset(from_matrix_to_preset(np.array([x])))
#     result = evaluate_presets(preset)
#     return (float(MSE(result, target_mfcc)),)

if __name__ == '__main__':
    # target_mfcc = evaluate_presets(load_parameters_file('target.json'))
    print(load_parameters_file('target.json'))
    # init_solution_mask = np.squeeze(from_preset_to_matrix(normalize_preset(load_parameters_file('init.json'))))

    optimizable_params = PARAM_NAMES

    x0 = np.zeros(len(optimizable_params))
    # x0 = np.array([1.7751533e-01, 2.8770146e-01, 5.3684968e-01, 7.9313725e-01,
    #            6.6560894e-01, 6.7737550e-01, 2.7434433e-01, 6.8054080e-02,
    #            7.9914108e-02, 4.7150740e-01, 6.7814648e-01, 2.7759978e-01,
    #            3.0605119e-01, 6.0443670e-01, 1.0590127e-01, 2.6214758e-02,
    #            2.5412971e-01, 6.5112638e-01, 5.7728428e-01, 5.2473575e-02,
    #            5.5560142e-01, 2.8625080e-01, 2.5980628e-01, 1.2211508e-02,
    #            6.0571535e-03, 5.6625915e-01, 3.1246966e-01, 5.2968508e-01,
    #            8.6832282e-05])
    sigma0 = 0.6

    # Crear estrategia
    es = cma.CMAEvolutionStrategy(x0, sigma0, {
        'popsize': 15, 
        'maxiter': 1000, 
        'bounds': [np.zeros_like(x0), np.ones_like(x0)],
        # 'minstd': 1e-3,
        'tolfun':1e-8, 
        # 'sigma_min':1e-1,
    })

    nh = cma.NoiseHandler(N=len(x0))  # N = número de variables

    gen = 1

    with Pool(PROCESSORS) as pool:
        while not es.stop():
            # Generar población
            solutions = np.array(es.ask(), dtype=np.float32)  # devuelve una lista de individuos

            # solutions_mask = np.tile(init_solution_mask, (solutions.shape[0], 1)) # Para logica de optimizacion por partes
            # for i, param in enumerate(optimizable_params):
            #     idx = PARAM_NAMES.index(param)
            #     solutions_mask[:,idx] = solutions[:,i]

            # s = render_presets(from_matrix_to_preset(solutions))

            # solutions es un array numpy o similar
            # if np.any(np.isnan(s)):
            #     print("¡Hay NaN en solutions!")
            # if np.any(np.isinf(s)):
            #     print("¡Hay Inf en solutions!")

            # # Alternativa combinada:
            # if not np.all(np.isfinite(s)):
            #     print("¡Hay valores no finitos en solutions!")

            fitnesses = get_fitness(solutions)
            

            best_idx = np.argmin(fitnesses)          # índice del mejor fitness
            best_solution = solutions[best_idx]      # solución correspondiente
            best_fitness = fitnesses[best_idx]       # fitness correspondiente

            print("Gen", gen, "Mejor fitness:", best_fitness, "Sigma", es.sigma)

            es.tell(solutions, fitnesses)  # pasar fitness al algoritmo

            # es.sigma *= nh(solutions, fitnesses, evaluate_single, es.ask)
            # es.countevals += nh.evaluations_just_done

            if best_fitness < 1.552335e-05:
                break
            
            # if es.sigma < 0.005:
            #     # es.sigma = max(es.sigma, 0.01)
            #     es.sigma = 0.1

            gen += 1

    # Mejor solución
    best_solution = es.result.xbest
    print("Mejor individuo:", best_solution)

{'osc1_shape': array([3.]), 'osc1_phase': array([0]), 'osc1_volume': array([1]), 'osc1_freq': array([7040]), 'osc2_shape': array([0]), 'osc2_phase': array([0]), 'osc2_volume': array([0]), 'osc2_freq': array([0]), 'osc3_shape': array([0]), 'osc3_phase': array([0]), 'osc3_volume': array([0]), 'osc3_freq': array([0]), 'osc4_shape': array([0]), 'osc4_phase': array([0]), 'osc4_volume': array([0]), 'osc4_freq': array([0]), 'oscnoise_volume': array([0]), 'base_cutoff_hz': array([0]), 'base_q': array([0.707]), 'filter_type': array([0]), 'envelope_depth': array([0]), 'cutoff_mod_depth': array([], dtype=float64), 'envelope_decay': array([0]), 'envelope_attack': array([0]), 'envelope_sustain': array([1]), 'envelope_release': array([0]), 'filter_envelope_attack': array([0]), 'filter_envelope_decay': array([0]), 'filter_envelope_sustain': array([0]), 'filter_envelope_release': array([0])}
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 29 (seed=599950, Wed Jan 14 20:27:28 2026)


c:\Users\bryan\Documents\GitHub\Synth\venv\Lib\site-packages\cma\evolution_strategy.py:1531: UserWarning: Sampling standard deviation i=0 (and 28 others) at iteration 0 multiplied by 0.5555555555555556 to stds[0]=0.3333333333333333
  warnings.warn("Sampling standard deviation i={0}{4} at iteration {1}"


Gen 1 Mejor fitness: 0.023193348 Sigma 0.6
Gen 2 Mejor fitness: 0.022636233 Sigma 0.6128804281706586
Gen 3 Mejor fitness: 0.021389173 Sigma 0.6283919278071104
Gen 4 Mejor fitness: 0.018402727 Sigma 0.6474060093669561
Gen 5 Mejor fitness: 0.019237082 Sigma 0.6604802028258707
Gen 6 Mejor fitness: 0.02223864 Sigma 0.677335438326015
Gen 7 Mejor fitness: 0.022604903 Sigma 0.682459413407221
Gen 8 Mejor fitness: 0.016271453 Sigma 0.6655436087592683
Gen 9 Mejor fitness: 0.022292381 Sigma 0.6388194781759925
Gen 10 Mejor fitness: 0.02006089 Sigma 0.619151582777986
Gen 11 Mejor fitness: 0.019908259 Sigma 0.586505692491079
Gen 12 Mejor fitness: 0.020874305 Sigma 0.577695613767111
Gen 13 Mejor fitness: 0.0180275 Sigma 0.5627152027188086
Gen 14 Mejor fitness: 0.01708765 Sigma 0.5406901112335732
Gen 15 Mejor fitness: 0.016669657 Sigma 0.5180525382250026
Gen 16 Mejor fitness: 0.013521919 Sigma 0.49266303406517853
Gen 17 Mejor fitness: 0.019642493 Sigma 0.4782480834428003
Gen 18 Mejor fitness: 0.015640

KeyboardInterrupt: 

In [ ]:
# from deap import base, creator, tools, algorithms
# import array
# import random

# IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY = len(PARAM_NAMES), 0, 1, 0, 1

# def checkStrategy(minstrategy):
#     def decorator(func):
#         def wrappper(*args, **kargs):
#             children = func(*args, **kargs)
#             for child in children:
#                 for i, s in enumerate(child.strategy):
#                     if s < minstrategy:
#                         child.strategy[i] = minstrategy
#             return children
#         return wrappper
#     return decorator

# def checkBounds(minval, maxval):
#     def decorator(func):
#         def wrapper(*args, **kwargs):
#             children = func(*args, **kwargs)
#             for child in children:
#                 for i in range(len(child)):
#                     if child[i] < minval:
#                         child[i] = minval
#                     elif child[i] > maxval:
#                         child[i] = maxval
#             return children
#         return wrapper
#     return decorator

# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
# creator.create("Strategy", array.array, typecode="d")

# def generateES(icls, scls, size, imin, imax, smin, smax):
#     ind = icls(random.uniform(imin, imax) for _ in range(size))
#     ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
#     return ind

# toolbox = base.Toolbox()

# toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
#     IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRATEGY, MAX_STRATEGY)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# toolbox.register("evaluate", evaluate_single)

# toolbox.register("mate", tools.cxESBlend, alpha=0.1)
# toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
# toolbox.register("select", tools.selTournament, tournsize=3)

# toolbox.decorate("mate", checkStrategy(MIN_STRATEGY))
# toolbox.decorate("mutate", checkStrategy(MIN_STRATEGY))

# toolbox.decorate("mate", checkBounds(MIN_VALUE, MAX_VALUE))
# toolbox.decorate("mutate", checkBounds(MIN_VALUE, MAX_VALUE))

# def main():
#     MU, LAMBDA = 10, 100
#     pop = toolbox.population(n=MU)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     # stats.register("avg", np.mean)
#     # stats.register("std", np.std)
#     # stats.register("min", np.min)
#     stats.register("max", np.max)

#     pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
#         cxpb=0.6, mutpb=0.3, ngen=500, stats=stats, halloffame=hof, verbose=True)

#     # return pop, logbook, hof
#     print('El mejor individuo es', hof[0], evaluate_single(hof[0]))

# main()

In [ ]:
# solution = init_solution_mask
# for i, param in enumerate(optimizable_params):
#     idx = PARAM_NAMES.index(param)
#     solution[idx] = best_solution[i]

solution = np.expand_dims(best_solution, axis=0)
error = evaluate_presets((denormalize_preset(from_matrix_to_preset(solution)), target_C))
print('Error', error)

target_audio = render_presets(load_parameters_file('target.json'))
predicted_audio = render_presets(denormalize_preset(from_matrix_to_preset(solution)))

Error [0.00606499]


In [ ]:
from scipy.io import wavfile

# NAME = "exports/s3_a8"

# audio_int16 = np.int16(target_audio[0] / np.max(np.abs(target_audio[0])) * 32767)
# wavfile.write(f"{NAME}_target.wav", SAMPLE_RATE, audio_int16)

Audio(target_audio[0], rate=SAMPLE_RATE)

In [ ]:
# audio_int16 = np.int16(predicted_audio[0] / np.max(np.abs(predicted_audio[0])) * 32767)
# wavfile.write(f"{NAME}_predicted_{error}.wav", SAMPLE_RATE, audio_int16)

Audio(predicted_audio[0], rate=SAMPLE_RATE)

In [ ]:
denormalized_predicted_preset = denormalize_preset(from_matrix_to_preset(solution))
pretty_print(denormalized_predicted_preset)

{
  osc1_shape:   [
    0.7100613117218018
  ]
  osc1_phase:   [
    0.28770145773887634
  ]
  osc1_volume:   [
    0.5368496775627136
  ]
  osc1_freq:   [
    5176.709536502857
  ]
  osc2_shape:   [
    2.66243577003479
  ]
  osc2_phase:   [
    0.6773754954338074
  ]
  osc2_volume:   [
    0.2743443250656128
  ]
  osc2_freq:   [
    32.21634535474818
  ]
  osc3_shape:   [
    0.3196564316749573
  ]
  osc3_phase:   [
    0.47150740027427673
  ]
  osc3_volume:   [
    0.678146481513977
  ]
  osc3_freq:   [
    139.82783407671008
  ]
  osc4_shape:   [
    1.2242047786712646
  ]
  osc4_phase:   [
    0.6044366955757141
  ]
  osc4_volume:   [
    0.10590127110481262
  ]
  osc4_freq:   [
    24.03173985899989
  ]
  oscnoise_volume:   [
    0.25412970781326294
  ]
  filter_envelope_attack:   [
    0.28312957286834717
  ]
  filter_envelope_decay:   [
    0.3124696612358093
  ]
  filter_envelope_sustain:   [
    0.5296850800514221
  ]
  filter_envelope_release:   [
    4.341614112490788e-05
 

In [ ]:
np.diagonal(es.C)

array([8.44323800e-01, 1.60477745e+00, 3.58118490e-01, 8.28203664e-04,
       6.57308094e-01, 1.46335381e+00, 1.00252050e+00, 8.55308654e-01,
       9.92309086e-01, 9.65197699e-01, 5.16891619e-01, 5.16977175e-01,
       9.29855335e-01, 6.13950347e-01, 7.60828207e-01, 8.45150136e-01,
       5.88128785e-01, 4.69393899e-01, 6.91347817e-01, 6.89698415e-01,
       7.28064587e-01, 6.87918098e-01, 9.64511423e-01, 5.69375764e-01,
       5.38398688e-01, 8.15663902e-01, 7.02019685e-01, 1.06870160e+00,
       8.87542476e-01])